# base

In [1]:
import vectorbt as vbt

from vectorbt.base import column_grouper, array_wrapper, combine_fns, index_fns, indexing, reshape_fns

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
from numba import njit
import itertools

In [3]:
v1 = 0
a1 = np.array([1])
a2 = np.array([1, 2, 3])
a3 = np.array([[1, 2, 3]])
a4 = np.array([[1], [2], [3]])
a5 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
sr_none = pd.Series([1])
print(sr_none)
sr1 = pd.Series([1], index=pd.Index(['x1'], name='i1'), name='a1')
print(sr1)
sr2 = pd.Series([1, 2, 3], index=pd.Index(['x2', 'y2', 'z2'], name='i2'), name='a2')
print(sr2)
df_none = pd.DataFrame([[1]])
print(df_none)
df1 = pd.DataFrame(
    [[1]], 
    index=pd.Index(['x3'], name='i3'), 
    columns=pd.Index(['a3'], name='c3'))
print(df1)
df2 = pd.DataFrame(
    [[1], [2], [3]], 
    index=pd.Index(['x4', 'y4', 'z4'], name='i4'), 
    columns=pd.Index(['a4'], name='c4'))
print(df2)
df3 = pd.DataFrame(
    [[1, 2, 3]], 
    index=pd.Index(['x5'], name='i5'), 
    columns=pd.Index(['a5', 'b5', 'c5'], name='c5'))
print(df3)
df4 = pd.DataFrame(
    [[1, 2, 3], [4, 5, 6], [7, 8, 9]], 
    index=pd.Index(['x6', 'y6', 'z6'], name='i6'), 
    columns=pd.Index(['a6', 'b6', 'c6'], name='c6'))
print(df4)

multi_i = pd.MultiIndex.from_arrays([['x7', 'y7', 'z7'], ['x8', 'y8', 'z8']], names=['i7', 'i8']) 
multi_c = pd.MultiIndex.from_arrays([['a7', 'b7', 'c7'], ['a8', 'b8', 'c8']], names=['c7', 'c8'])
df5 = pd.DataFrame([[1, 2, 3], [4, 5, 6], [7, 8, 9]], index=multi_i, columns=multi_c)
print(df5)

0    1
dtype: int64
i1
x1    1
Name: a1, dtype: int64
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64
   0
0  1
c3  a3
i3    
x3   1
c4  a4
i4    
x4   1
y4   2
z4   3
c5  a5  b5  c5
i5            
x5   1   2   3
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  2  3
y7 y8  4  5  6
z7 z8  7  8  9


## column_grouper

In [4]:
some_columns = pd.MultiIndex.from_arrays([
    [1, 1, 1, 1, 0, 0, 0, 0],
    [3, 3, 2, 2, 1, 1, 0, 0],
    [7, 6, 5, 4, 3, 2, 1, 0]
], names=['first', 'second', 'third'])

In [5]:
print(column_grouper.group_by_to_index(some_columns, group_by=0))
print(column_grouper.group_by_to_index(some_columns, group_by='first'))
print(column_grouper.group_by_to_index(some_columns, group_by=[0, 1]))
print(column_grouper.group_by_to_index(some_columns, group_by=['first', 'second']))
print(column_grouper.group_by_to_index(some_columns, group_by=np.array([3, 2, 1, 1, 1, 0, 0, 0])))
print(column_grouper.group_by_to_index(some_columns, group_by=pd.Index([3, 2, 1, 1, 1, 0, 0, 0], name='fourth')))

Index([1, 1, 1, 1, 0, 0, 0, 0], dtype='int64', name='first')
Index([1, 1, 1, 1, 0, 0, 0, 0], dtype='int64', name='first')
MultiIndex([(1, 3),
            (1, 3),
            (1, 2),
            (1, 2),
            (0, 1),
            (0, 1),
            (0, 0),
            (0, 0)],
           names=['first', 'second'])
MultiIndex([(1, 3),
            (1, 3),
            (1, 2),
            (1, 2),
            (0, 1),
            (0, 1),
            (0, 0),
            (0, 0)],
           names=['first', 'second'])
Index([3, 2, 1, 1, 1, 0, 0, 0], dtype='int64')
Index([3, 2, 1, 1, 1, 0, 0, 0], dtype='int64', name='fourth')


In [6]:
# group_arr comes always from 0 to n, also keeps order
print(column_grouper.get_groups_and_index(some_columns, 0))
print(column_grouper.get_groups_and_index(some_columns, [0, 1]))
print(column_grouper.get_groups_and_index(some_columns, np.array([3, 2, 1, 1, 1, 0, 0, 0])))

(array([0, 0, 0, 0, 1, 1, 1, 1]), Index([1, 0], dtype='int64', name='first'))
(array([0, 0, 1, 1, 2, 2, 3, 3]), MultiIndex([(1, 3),
            (1, 2),
            (0, 1),
            (0, 0)],
           names=['first', 'second']))
(array([0, 1, 2, 2, 2, 3, 3, 3]), Index([3, 2, 1, 0], dtype='int64'))


In [7]:
print(column_grouper.get_group_lens_nb(np.array([0, 0, 0, 0, 1, 1, 1, 1])))
print(column_grouper.get_group_lens_nb(np.array([0, 1])))
print(column_grouper.get_group_lens_nb(np.array([0, 0])))
print(column_grouper.get_group_lens_nb(np.array([0])))
print(column_grouper.get_group_lens_nb(np.array([])))

[4 4]
[1 1]
[2]
[1]
[]


In [8]:
print(column_grouper.ColumnGrouper(sr2.to_frame().columns, group_by=np.array([0])).group_by)
print(column_grouper.ColumnGrouper(sr2.to_frame().columns, group_by=np.array([0])).get_groups_and_columns())
print(column_grouper.ColumnGrouper(sr2.to_frame().columns, group_by=np.array([0])).get_groups())
print(column_grouper.ColumnGrouper(sr2.to_frame().columns, group_by=np.array([0])).get_columns())
print(column_grouper.ColumnGrouper(sr2.to_frame().columns, group_by=np.array([0])).get_group_lens())
print(column_grouper.ColumnGrouper(sr2.to_frame().columns, group_by=np.array([0])).get_group_start_idxs())
print(column_grouper.ColumnGrouper(sr2.to_frame().columns, group_by=np.array([0])).get_group_end_idxs())

Index([0], dtype='int64')
(array([0]), Index([0], dtype='int64'))
[0]
Index([0], dtype='int64')
[1]
[0]
[1]


In [9]:
print(column_grouper.ColumnGrouper(df4.columns, group_by=np.array([0, 0, 1])).group_by)
print(column_grouper.ColumnGrouper(df4.columns, group_by=np.array([0, 0, 1])).get_groups_and_columns())
print(column_grouper.ColumnGrouper(df4.columns, group_by=np.array([0, 0, 1])).get_groups())
print(column_grouper.ColumnGrouper(df4.columns, group_by=np.array([0, 0, 1])).get_columns())
print(column_grouper.ColumnGrouper(df4.columns, group_by=np.array([0, 0, 1])).get_group_lens())
print(column_grouper.ColumnGrouper(df4.columns, group_by=np.array([0, 0, 1])).get_group_start_idxs())
print(column_grouper.ColumnGrouper(df4.columns, group_by=np.array([0, 0, 1])).get_group_end_idxs())

Index([0, 0, 1], dtype='int64')
(array([0, 0, 1]), Index([0, 1], dtype='int64'))
[0 0 1]
Index([0, 1], dtype='int64')
[2 1]
[0 2]
[2 3]


## array_wrapper

In [10]:
sr2_wrapper = array_wrapper.ArrayWrapper.from_obj(sr2)
df4_wrapper = array_wrapper.ArrayWrapper.from_obj(df4)

sr2_wrapper_co = sr2_wrapper.copy(column_only_select=True)
df4_wrapper_co = df4_wrapper.copy(column_only_select=True)

sr2_grouped_wrapper = sr2_wrapper.copy(group_by=np.array([0]))
df4_grouped_wrapper = df4_wrapper.copy(group_by=np.array([0, 0, 1]))

sr2_grouped_wrapper_co = sr2_grouped_wrapper.copy(column_only_select=True)
df4_grouped_wrapper_co = df4_grouped_wrapper.copy(column_only_select=True)

In [11]:
# test indexing
print(sr2_wrapper.indexing_func_meta(lambda x: x.iloc[:2])[1:])
print(df4_wrapper.indexing_func_meta(lambda x: x.iloc[0, :2])[1:])
print(df4_wrapper.indexing_func_meta(lambda x: x.iloc[:2, 0])[1:])
print(df4_wrapper.indexing_func_meta(lambda x: x.iloc[:2, [0]])[1:])
print(df4_wrapper.indexing_func_meta(lambda x: x.iloc[:2, :2])[1:])

(array([0, 1]), 0, 0)
(0, array([0, 1]), array([0, 1]))
(array([0, 1]), 0, 0)
(array([0, 1]), array([0]), array([0]))
(array([0, 1]), array([0, 1]), array([0, 1]))


In [12]:
print(df4_wrapper_co.indexing_func_meta(lambda x: x.iloc[0])[1:])
print(df4_wrapper_co.indexing_func_meta(lambda x: x.iloc[[0]])[1:])
print(df4_wrapper_co.indexing_func_meta(lambda x: x.iloc[:2])[1:])

(array([0, 1, 2]), 0, 0)
(array([0, 1, 2]), array([0]), array([0]))
(array([0, 1, 2]), array([0, 1]), array([0, 1]))


In [13]:
print(sr2_grouped_wrapper.indexing_func_meta(lambda x: x.iloc[:2])[1:])
print(df4_grouped_wrapper.indexing_func_meta(lambda x: x.iloc[:2, 0])[1:])
print(df4_grouped_wrapper.indexing_func_meta(lambda x: x.iloc[:2, 1])[1:])
print(df4_grouped_wrapper.indexing_func_meta(lambda x: x.iloc[:2, [1]])[1:])
print(df4_grouped_wrapper.indexing_func_meta(lambda x: x.iloc[:2, :2])[1:])

(array([0, 1]), 0, 0)
(array([0, 1]), 0, array([0, 1]))
(array([0, 1]), 1, 2)
(array([0, 1]), array([1]), array([2]))
(array([0, 1]), array([0, 1]), array([0, 1, 2]))


In [14]:
print(df4_grouped_wrapper_co.indexing_func_meta(lambda x: x.iloc[0])[1:])
print(df4_grouped_wrapper_co.indexing_func_meta(lambda x: x.iloc[1])[1:])
print(df4_grouped_wrapper_co.indexing_func_meta(lambda x: x.iloc[[1]])[1:])
print(df4_grouped_wrapper_co.indexing_func_meta(lambda x: x.iloc[:2])[1:])

(array([0, 1, 2]), 0, array([0, 1]))
(array([0, 1, 2]), 1, 2)
(array([0, 1, 2]), array([1]), array([2]))
(array([0, 1, 2]), array([0, 1]), array([0, 1, 2]))


In [15]:
print(sr2_wrapper.iloc[:2].index)
print(sr2_wrapper.iloc[:2].columns)
print(sr2_wrapper.iloc[:2].ndim)

print(df4_wrapper.iloc[0, :2].index)
print(df4_wrapper.iloc[0, :2].columns)
print(df4_wrapper.iloc[0, :2].ndim)

print(df4_wrapper.iloc[:2, 0].index)
print(df4_wrapper.iloc[:2, 0].columns)
print(df4_wrapper.iloc[:2, 0].ndim)

print(df4_wrapper.iloc[:2, [0]].index)
print(df4_wrapper.iloc[:2, [0]].columns)
print(df4_wrapper.iloc[:2, [0]].ndim)

print(df4_wrapper.iloc[:2, :2].index)
print(df4_wrapper.iloc[:2, :2].columns)
print(df4_wrapper.iloc[:2, :2].ndim)

Index(['x2', 'y2'], dtype='object', name='i2')
Index(['a2'], dtype='object')
1
Index(['a6', 'b6'], dtype='object', name='c6')
Index(['x6'], dtype='object', name='i6')
1
Index(['x6', 'y6'], dtype='object', name='i6')
Index(['a6'], dtype='object', name='c6')
1
Index(['x6', 'y6'], dtype='object', name='i6')
Index(['a6'], dtype='object', name='c6')
2
Index(['x6', 'y6'], dtype='object', name='i6')
Index(['a6', 'b6'], dtype='object', name='c6')
2


In [16]:
print(df4_wrapper_co.iloc[0].index)
print(df4_wrapper_co.iloc[0].columns)
print(df4_wrapper_co.iloc[0].ndim)

print(df4_wrapper_co.iloc[[0]].index)
print(df4_wrapper_co.iloc[[0]].columns)
print(df4_wrapper_co.iloc[[0]].ndim)

print(df4_wrapper_co.iloc[:2].index)
print(df4_wrapper_co.iloc[:2].columns)
print(df4_wrapper_co.iloc[:2].ndim)

Index(['x6', 'y6', 'z6'], dtype='object', name='i6')
Index(['a6'], dtype='object', name='c6')
1
Index(['x6', 'y6', 'z6'], dtype='object', name='i6')
Index(['a6'], dtype='object', name='c6')
2
Index(['x6', 'y6', 'z6'], dtype='object', name='i6')
Index(['a6', 'b6'], dtype='object', name='c6')
2


In [17]:
print(sr2_grouped_wrapper.iloc[:2].index)
print(sr2_grouped_wrapper.iloc[:2].columns)
print(sr2_grouped_wrapper.iloc[:2].ndim)
print(sr2_grouped_wrapper.iloc[:2].grouped_ndim)
print(sr2_grouped_wrapper.iloc[:2].grouper.group_by)

print(df4_grouped_wrapper.iloc[:2, 0].index)
print(df4_grouped_wrapper.iloc[:2, 0].columns)
print(df4_grouped_wrapper.iloc[:2, 0].ndim)
print(df4_grouped_wrapper.iloc[:2, 0].grouped_ndim)
print(df4_grouped_wrapper.iloc[:2, 0].grouper.group_by)

print(df4_grouped_wrapper.iloc[:2, 1].index)
print(df4_grouped_wrapper.iloc[:2, 1].columns)
print(df4_grouped_wrapper.iloc[:2, 1].ndim)
print(df4_grouped_wrapper.iloc[:2, 1].grouped_ndim)
print(df4_grouped_wrapper.iloc[:2, 1].grouper.group_by)

print(df4_grouped_wrapper.iloc[:2, [1]].index)
print(df4_grouped_wrapper.iloc[:2, [1]].columns)
print(df4_grouped_wrapper.iloc[:2, [1]].ndim)
print(df4_grouped_wrapper.iloc[:2, [1]].grouped_ndim)
print(df4_grouped_wrapper.iloc[:2, [1]].grouper.group_by)

print(df4_grouped_wrapper.iloc[:2, :2].index)
print(df4_grouped_wrapper.iloc[:2, :2].columns)
print(df4_grouped_wrapper.iloc[:2, :2].ndim)
print(df4_grouped_wrapper.iloc[:2, :2].grouped_ndim)
print(df4_grouped_wrapper.iloc[:2, :2].grouper.group_by)

Index(['x2', 'y2'], dtype='object', name='i2')
Index(['a2'], dtype='object')
1
1
Index([0], dtype='int64')
Index(['x6', 'y6'], dtype='object', name='i6')
Index(['a6', 'b6'], dtype='object', name='c6')
2
1
Index([0, 0], dtype='int64')
Index(['x6', 'y6'], dtype='object', name='i6')
Index(['c6'], dtype='object', name='c6')
1
1
Index([1], dtype='int64')
Index(['x6', 'y6'], dtype='object', name='i6')
Index(['c6'], dtype='object', name='c6')
2
2
Index([1], dtype='int64')
Index(['x6', 'y6'], dtype='object', name='i6')
Index(['a6', 'b6', 'c6'], dtype='object', name='c6')
2
2
Index([0, 0, 1], dtype='int64')


In [18]:
print(df4_grouped_wrapper_co.iloc[0].index)
print(df4_grouped_wrapper_co.iloc[0].columns)
print(df4_grouped_wrapper_co.iloc[0].ndim)
print(df4_grouped_wrapper_co.iloc[0].grouped_ndim)
print(df4_grouped_wrapper_co.iloc[0].grouper.group_by)

print(df4_grouped_wrapper_co.iloc[1].index)
print(df4_grouped_wrapper_co.iloc[1].columns)
print(df4_grouped_wrapper_co.iloc[1].ndim)
print(df4_grouped_wrapper_co.iloc[1].grouped_ndim)
print(df4_grouped_wrapper_co.iloc[1].grouper.group_by)

print(df4_grouped_wrapper_co.iloc[[1]].index)
print(df4_grouped_wrapper_co.iloc[[1]].columns)
print(df4_grouped_wrapper_co.iloc[[1]].ndim)
print(df4_grouped_wrapper_co.iloc[[1]].grouped_ndim)
print(df4_grouped_wrapper_co.iloc[[1]].grouper.group_by)

print(df4_grouped_wrapper_co.iloc[:2].index)
print(df4_grouped_wrapper_co.iloc[:2].columns)
print(df4_grouped_wrapper_co.iloc[:2].ndim)
print(df4_grouped_wrapper_co.iloc[:2].grouped_ndim)
print(df4_grouped_wrapper_co.iloc[:2].grouper.group_by)

Index(['x6', 'y6', 'z6'], dtype='object', name='i6')
Index(['a6', 'b6'], dtype='object', name='c6')
2
1
Index([0, 0], dtype='int64')
Index(['x6', 'y6', 'z6'], dtype='object', name='i6')
Index(['c6'], dtype='object', name='c6')
1
1
Index([1], dtype='int64')
Index(['x6', 'y6', 'z6'], dtype='object', name='i6')
Index(['c6'], dtype='object', name='c6')
2
2
Index([1], dtype='int64')
Index(['x6', 'y6', 'z6'], dtype='object', name='i6')
Index(['a6', 'b6', 'c6'], dtype='object', name='c6')
2
2
Index([0, 0, 1], dtype='int64')


In [19]:
big_df = pd.DataFrame(np.empty((1000, 1000)))

big_df_wrapper = array_wrapper.ArrayWrapper.from_obj(big_df)
big_df_wrapper_co = big_df_wrapper.copy(column_only_select=True)
big_df_grouped_wrapper = df4_wrapper.copy(group_by=np.array([0, 0, 1]))
big_df_grouped_wrapper_co = big_df_grouped_wrapper.copy(column_only_select=True)

In [20]:
%timeit big_df_wrapper.iloc[:, 0]
%timeit big_df_wrapper.iloc[:, :]

%timeit big_df_wrapper_co.iloc[0]
%timeit big_df_wrapper_co.iloc[:]

%timeit big_df_grouped_wrapper.iloc[:, 0]
%timeit big_df_grouped_wrapper.iloc[:, :]

%timeit big_df_grouped_wrapper_co.iloc[0]
%timeit big_df_grouped_wrapper_co.iloc[:]

759 µs ± 10.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
680 µs ± 15.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
569 µs ± 8.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
579 µs ± 6.89 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
839 µs ± 4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
748 µs ± 7.08 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
648 µs ± 6.35 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
658 µs ± 4.24 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
print(df4_grouped_wrapper_co.wrap(np.array([[1, 2], [3, 4], [5, 6]])))
print(df4_grouped_wrapper_co.wrap_reduced(np.array([1, 2])))

print(df4_grouped_wrapper_co.wrap(np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]), group_by=False))
print(df4_grouped_wrapper_co.wrap_reduced(np.array([1, 2, 3]), group_by=False))

    0  1
i6      
x6  1  2
y6  3  4
z6  5  6
0    1
1    2
dtype: int64
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
c6
a6    1
b6    2
c6    3
dtype: int64


In [22]:
print(df4_grouped_wrapper_co.iloc[0].wrap(np.array([1, 2, 3])))
print(df4_grouped_wrapper_co.iloc[0].wrap_reduced(np.array([1])))

print(df4_grouped_wrapper_co.iloc[0].wrap(np.array([[1, 2], [3, 4], [5, 6]]), group_by=False))
print(df4_grouped_wrapper_co.iloc[0].wrap_reduced(np.array([1, 2]), group_by=False))

i6
x6    1
y6    2
z6    3
dtype: int64
1
c6  a6  b6
i6        
x6   1   2
y6   3   4
z6   5   6
c6
a6    1
b6    2
dtype: int64


In [23]:
print(df4_grouped_wrapper_co.iloc[[0]].wrap(np.array([1, 2, 3])))
print(df4_grouped_wrapper_co.iloc[[0]].wrap_reduced(np.array([1])))

print(df4_grouped_wrapper_co.iloc[[0]].wrap(np.array([[1, 2], [3, 4], [5, 6]]), group_by=False))
print(df4_grouped_wrapper_co.iloc[[0]].wrap_reduced(np.array([1, 2]), group_by=False))

    0
i6   
x6  1
y6  2
z6  3
0    1
dtype: int64
c6  a6  b6
i6        
x6   1   2
y6   3   4
z6   5   6
c6
a6    1
b6    2
dtype: int64


In [24]:
print(df4_grouped_wrapper_co.iloc[1].wrap(np.array([1, 2, 3])))
print(df4_grouped_wrapper_co.iloc[1].wrap_reduced(np.array([1])))

print(df4_grouped_wrapper_co.iloc[1].wrap(np.array([1, 2, 3]), group_by=False))
print(df4_grouped_wrapper_co.iloc[1].wrap_reduced(np.array([1]), group_by=False))

i6
x6    1
y6    2
z6    3
Name: 1, dtype: int64
1
i6
x6    1
y6    2
z6    3
Name: c6, dtype: int64
1


In [25]:
print(df4_grouped_wrapper_co.iloc[[1]].wrap(np.array([1, 2, 3])))
print(df4_grouped_wrapper_co.iloc[[1]].wrap_reduced(np.array([1])))

print(df4_grouped_wrapper_co.iloc[[1]].wrap(np.array([1, 2, 3]), group_by=False))
print(df4_grouped_wrapper_co.iloc[[1]].wrap_reduced(np.array([1]), group_by=False))

    1
i6   
x6  1
y6  2
z6  3
1    1
dtype: int64
c6  c6
i6    
x6   1
y6   2
z6   3
c6
c6    1
dtype: int64


## index_fns

In [26]:
i1 = index_fns.index_from_values([0.1, 0.2], name='a')
i2 = index_fns.index_from_values(np.tile(np.arange(1, 4)[:, None][:, None], (1, 3, 3)), name='b')
i3 = index_fns.index_from_values(np.random.uniform(size=(3, 3, 3)), name='c')

print(i1)
print(i2)
print(i3)

Index([0.1, 0.2], dtype='float64', name='a')
Index([1, 2, 3], dtype='int64', name='b')
Index(['array_0', 'array_1', 'array_2'], dtype='object', name='c')


In [27]:
print(index_fns.repeat_index(i2, 3))
print(index_fns.repeat_index(multi_i, 3))

Index([1, 1, 1, 2, 2, 2, 3, 3, 3], dtype='int64', name='b')
MultiIndex([('x7', 'x8'),
            ('x7', 'x8'),
            ('x7', 'x8'),
            ('y7', 'y8'),
            ('y7', 'y8'),
            ('y7', 'y8'),
            ('z7', 'z8'),
            ('z7', 'z8'),
            ('z7', 'z8')],
           names=['i7', 'i8'])


In [28]:
print(index_fns.tile_index(i2, 3))
print(index_fns.tile_index(multi_i, 3))

Index([1, 2, 3, 1, 2, 3, 1, 2, 3], dtype='int64', name='b')
MultiIndex([('x7', 'x8'),
            ('y7', 'y8'),
            ('z7', 'z8'),
            ('x7', 'x8'),
            ('y7', 'y8'),
            ('z7', 'z8'),
            ('x7', 'x8'),
            ('y7', 'y8'),
            ('z7', 'z8')],
           names=['i7', 'i8'])


In [29]:
i23 = index_fns.stack_indexes((i2, i3))
i32 = index_fns.stack_indexes((i3, i2))

print(i23)
print(i32)

print(index_fns.stack_indexes((multi_i, multi_i), drop_duplicates=False))
print(index_fns.stack_indexes((multi_i, multi_i), drop_duplicates=True))
print(index_fns.stack_indexes(([0, 1], ['a', 'b']), drop_redundant=False))
print(index_fns.stack_indexes(([0, 1], ['a', 'b']), drop_redundant=True))
print(index_fns.stack_indexes((pd.Index([0, 1], name='test_name'), ['a', 'b']), drop_redundant=True))
print(index_fns.stack_indexes((['a', 'a'], ['a', 'b']), drop_redundant=True))
print(index_fns.stack_indexes((pd.Index(['a', 'a'], name='test_name'), ['a', 'b']), drop_redundant=True))

MultiIndex([(1, 'array_0'),
            (2, 'array_1'),
            (3, 'array_2')],
           names=['b', 'c'])
MultiIndex([('array_0', 1),
            ('array_1', 2),
            ('array_2', 3)],
           names=['c', 'b'])
MultiIndex([('x7', 'x8', 'x7', 'x8'),
            ('y7', 'y8', 'y7', 'y8'),
            ('z7', 'z8', 'z7', 'z8')],
           names=['i7', 'i8', 'i7', 'i8'])
MultiIndex([('x7', 'x8'),
            ('y7', 'y8'),
            ('z7', 'z8')],
           names=['i7', 'i8'])
MultiIndex([(0, 'a'),
            (1, 'b')],
           )
Index(['a', 'b'], dtype='object')
MultiIndex([(0, 'a'),
            (1, 'b')],
           names=['test_name', None])
Index(['a', 'b'], dtype='object')
MultiIndex([('a', 'a'),
            ('a', 'b')],
           names=['test_name', None])


In [30]:
print(index_fns.combine_indexes((pd.Index([1]), pd.Index([2, 3])), drop_duplicates=False))
print(index_fns.combine_indexes((pd.Index([1]), pd.Index([2, 3])), drop_duplicates=True))
print(index_fns.combine_indexes((pd.Index([1, 2]), pd.Index([3])), drop_duplicates=False))
print(index_fns.combine_indexes((pd.Index([1, 2]), pd.Index([3])), drop_duplicates=True))
print(index_fns.combine_indexes((i1, i2))) # combine_fns uses stack
print(index_fns.combine_indexes((i2, i3)))
print(index_fns.combine_indexes((i23, i23)))

Index([2, 3], dtype='int64')
Index([2, 3], dtype='int64')
Index([1, 2], dtype='int64')
Index([1, 2], dtype='int64')
MultiIndex([(0.1, 1),
            (0.1, 2),
            (0.1, 3),
            (0.2, 1),
            (0.2, 2),
            (0.2, 3)],
           names=['a', 'b'])
MultiIndex([(1, 'array_0'),
            (1, 'array_1'),
            (1, 'array_2'),
            (2, 'array_0'),
            (2, 'array_1'),
            (2, 'array_2'),
            (3, 'array_0'),
            (3, 'array_1'),
            (3, 'array_2')],
           names=['b', 'c'])
MultiIndex([(1, 'array_0', 1, 'array_0'),
            (1, 'array_0', 2, 'array_1'),
            (1, 'array_0', 3, 'array_2'),
            (2, 'array_1', 1, 'array_0'),
            (2, 'array_1', 2, 'array_1'),
            (2, 'array_1', 3, 'array_2'),
            (3, 'array_2', 1, 'array_0'),
            (3, 'array_2', 2, 'array_1'),
            (3, 'array_2', 3, 'array_2')],
           names=['b', 'c', 'b', 'c'])


In [31]:
print(index_fns.drop_levels(multi_i, 'i10'))
print(index_fns.drop_levels(multi_i, ['i7', 'i8']))

MultiIndex([('x7', 'x8'),
            ('y7', 'y8'),
            ('z7', 'z8')],
           names=['i7', 'i8'])
MultiIndex([('x7', 'x8'),
            ('y7', 'y8'),
            ('z7', 'z8')],
           names=['i7', 'i8'])


In [32]:
print(index_fns.rename_levels(pd.Index([1, 2, 3], name='i'), {'i': 'f'}))
print(index_fns.rename_levels(multi_i, {'i7': 'f7', 'i8': 'f8'}))

Index([1, 2, 3], dtype='int64', name='f')
MultiIndex([('x7', 'x8'),
            ('y7', 'y8'),
            ('z7', 'z8')],
           names=['f7', 'f8'])


In [33]:
print(index_fns.select_levels(multi_i, 'i7'))
print(index_fns.select_levels(multi_i, ['i7']))
print(index_fns.select_levels(multi_i, ['i7', 'i8']))

Index(['x7', 'y7', 'z7'], dtype='object', name='i7')
MultiIndex([('x7',),
            ('y7',),
            ('z7',)],
           names=['i7'])
MultiIndex([('x7', 'x8'),
            ('y7', 'y8'),
            ('z7', 'z8')],
           names=['i7', 'i8'])


In [34]:
print(index_fns.drop_redundant_levels(pd.Index(['a', 'a']))) # ignores levels with single element
print(index_fns.drop_redundant_levels(pd.Index(['a', 'a'], name='hi')))
print(index_fns.drop_redundant_levels(pd.MultiIndex.from_arrays([['a', 'a'], ['b', 'b']], names=['hi', 'hi2'])))
print(index_fns.drop_redundant_levels(pd.MultiIndex.from_arrays([['a', 'b'], ['a', 'b']], names=['hi', 'hi2'])))
print(index_fns.drop_redundant_levels(pd.MultiIndex.from_arrays([[0, 1], ['a', 'b']], names=[None, 'hi2']))) # ignores 0-to-n
print(index_fns.drop_redundant_levels(pd.MultiIndex.from_arrays([[0, 2], ['a', 'b']], names=[None, 'hi2']))) # legit
print(index_fns.drop_redundant_levels(pd.MultiIndex.from_arrays([[0, 1], ['a', 'b']], names=['hi', 'hi2']))) # legit (w/ name)

Index(['a', 'a'], dtype='object')
Index(['a', 'a'], dtype='object', name='hi')
MultiIndex([('a', 'b'),
            ('a', 'b')],
           names=['hi', 'hi2'])
MultiIndex([('a', 'a'),
            ('b', 'b')],
           names=['hi', 'hi2'])
Index(['a', 'b'], dtype='object', name='hi2')
MultiIndex([(0, 'a'),
            (2, 'b')],
           names=[None, 'hi2'])
MultiIndex([(0, 'a'),
            (1, 'b')],
           names=['hi', 'hi2'])


In [35]:
print(index_fns.drop_duplicate_levels(pd.MultiIndex.from_arrays(
    [[1, 2, 3], [1, 2, 3]], names=['a', 'a'])))
print(index_fns.drop_duplicate_levels(pd.MultiIndex.from_tuples(
    [(0, 1, 2, 1), ('a', 'b', 'c', 'b')], names=['x', 'y', 'z', 'y']), keep='last'))
print(index_fns.drop_duplicate_levels(pd.MultiIndex.from_tuples(
    [(0, 1, 2, 1), ('a', 'b', 'c', 'b')], names=['x', 'y', 'z', 'y']), keep='first'))

Index([1, 2, 3], dtype='int64', name='a')
MultiIndex([(  0,   2,   1),
            ('a', 'c', 'b')],
           names=['x', 'z', 'y'])
MultiIndex([(  0,   1,   2),
            ('a', 'b', 'c')],
           names=['x', 'y', 'z'])


In [36]:
multi_c1 = pd.MultiIndex.from_arrays([['a8', 'b8']], names=['c8'])
multi_c2 = pd.MultiIndex.from_arrays([['a7', 'a7', 'c7', 'c7'], ['a8', 'b8', 'a8', 'b8']], names=['c7', 'c8'])

index_fns.align_index_to(multi_c1, multi_c2)

array([0, 1, 0, 1])

In [37]:
print(index_fns.pick_levels(multi_c, required_levels=[], optional_levels=[]))
print(index_fns.pick_levels(multi_c, required_levels=['c8'], optional_levels=[]))
print(index_fns.pick_levels(multi_c, required_levels=['c8'], optional_levels=[]))
print(index_fns.pick_levels(multi_c, required_levels=['c7', 'c8'], optional_levels=[]))
print(index_fns.pick_levels(multi_c, required_levels=['c8', None], optional_levels=[]))
print(index_fns.pick_levels(multi_c, required_levels=[None, None], optional_levels=[]))
print(index_fns.pick_levels(multi_c, required_levels=[None], optional_levels=['c8']))
print(index_fns.pick_levels(multi_c, required_levels=['c8'], optional_levels=[None]))
print(index_fns.pick_levels(multi_c, required_levels=[], optional_levels=['c7', 'c8']))

([], [])
([1], [])
([1], [])
([0, 1], [])
([1, 0], [])
([0, 1], [])
([0], [1])
([1], [None])
([], [0, 1])


## reshape_fns

In [38]:
print(reshape_fns.soft_to_ndim(a2, 1))
print(reshape_fns.soft_to_ndim(sr2, 1))
print(reshape_fns.soft_to_ndim(df2, 1))
print(reshape_fns.soft_to_ndim(df4, 1)) # cannot -> do nothing
print(reshape_fns.soft_to_ndim(a2, 2))
print(reshape_fns.soft_to_ndim(sr2, 2))
print(reshape_fns.soft_to_ndim(df2, 2))

[1 2 3]
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64
i4
x4    1
y4    2
z4    3
Name: a4, dtype: int64
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
[[1]
 [2]
 [3]]
    a2
i2    
x2   1
y2   2
z2   3
c4  a4
i4    
x4   1
y4   2
z4   3


In [39]:
print(reshape_fns.to_1d(None))
print(reshape_fns.to_1d(v1))
print(reshape_fns.to_1d(a1))
print(reshape_fns.to_1d(a2))
print(reshape_fns.to_1d(sr1))
print(reshape_fns.to_1d(sr2))
print(reshape_fns.to_1d(df1))
print(reshape_fns.to_1d(df2))

[None]
[0]
[1]
[1 2 3]
i1
x1    1
Name: a1, dtype: int64
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64
i3
x3    1
Name: a3, dtype: int64
i4
x4    1
y4    2
z4    3
Name: a4, dtype: int64


In [40]:
print(reshape_fns.to_2d(None))
print(reshape_fns.to_2d(v1))
print(reshape_fns.to_2d(a1))
print(reshape_fns.to_2d(a2))
print(reshape_fns.to_2d(sr1))
print(reshape_fns.to_2d(sr2))
print(reshape_fns.to_2d(sr2, expand_axis=0))

[[None]]
[[0]]
[[1]]
[[1]
 [2]
 [3]]
    a1
i1    
x1   1
    a2
i2    
x2   1
y2   2
z2   3
i2  x2  y2  z2
0    1   2   3


In [41]:
print(reshape_fns.repeat(v1, 3, axis=0))
print(reshape_fns.repeat(a1, 3, axis=0))
print(reshape_fns.repeat(a2, 3, axis=0))
print(reshape_fns.repeat(a3, 3, axis=0))
print(reshape_fns.repeat(a4, 3, axis=0))
print(reshape_fns.repeat(a5, 3, axis=0))
print(reshape_fns.repeat(sr_none, 3, axis=0))
print(reshape_fns.repeat(sr1, 3, axis=0))
print(reshape_fns.repeat(sr2, 3, axis=0))
print(reshape_fns.repeat(df_none, 3, axis=0))
print(reshape_fns.repeat(df1, 3, axis=0))
print(reshape_fns.repeat(df2, 3, axis=0))
print(reshape_fns.repeat(df3, 3, axis=0))
print(reshape_fns.repeat(df4, 3, axis=0))

[0 0 0]
[1 1 1]
[1 1 1 2 2 2 3 3 3]
[[1 2 3]
 [1 2 3]
 [1 2 3]]
[[1]
 [1]
 [1]
 [2]
 [2]
 [2]
 [3]
 [3]
 [3]]
[[1 2 3]
 [1 2 3]
 [1 2 3]
 [4 5 6]
 [4 5 6]
 [4 5 6]
 [7 8 9]
 [7 8 9]
 [7 8 9]]
0    1
1    1
2    1
dtype: int64
i1
x1    1
x1    1
x1    1
Name: a1, dtype: int64
i2
x2    1
x2    1
x2    1
y2    2
y2    2
y2    2
z2    3
z2    3
z2    3
Name: a2, dtype: int64
   0
0  1
1  1
2  1
c3  a3
i3    
x3   1
x3   1
x3   1
c4  a4
i4    
x4   1
x4   1
x4   1
y4   2
y4   2
y4   2
z4   3
z4   3
z4   3
c5  a5  b5  c5
i5            
x5   1   2   3
x5   1   2   3
x5   1   2   3
c6  a6  b6  c6
i6            
x6   1   2   3
x6   1   2   3
x6   1   2   3
y6   4   5   6
y6   4   5   6
y6   4   5   6
z6   7   8   9
z6   7   8   9
z6   7   8   9


In [42]:
print(reshape_fns.repeat(v1, 3, axis=1))
print(reshape_fns.repeat(a1, 3, axis=1))
print(reshape_fns.repeat(a2, 3, axis=1))
print(reshape_fns.repeat(a3, 3, axis=1))
print(reshape_fns.repeat(a4, 3, axis=1))
print(reshape_fns.repeat(a5, 3, axis=1))
print(reshape_fns.repeat(sr_none, 3, axis=1))
print(reshape_fns.repeat(sr1, 3, axis=1))
print(reshape_fns.repeat(sr2, 3, axis=1))
print(reshape_fns.repeat(df_none, 3, axis=1))
print(reshape_fns.repeat(df1, 3, axis=1))
print(reshape_fns.repeat(df2, 3, axis=1))
print(reshape_fns.repeat(df3, 3, axis=1))
print(reshape_fns.repeat(df4, 3, axis=1))

[[0 0 0]]
[[1 1 1]]
[[1 1 1]
 [2 2 2]
 [3 3 3]]
[[1 1 1 2 2 2 3 3 3]]
[[1 1 1]
 [2 2 2]
 [3 3 3]]
[[1 1 1 2 2 2 3 3 3]
 [4 4 4 5 5 5 6 6 6]
 [7 7 7 8 8 8 9 9 9]]
   0  1  2
0  1  1  1
    a1  a1  a1
i1            
x1   1   1   1
    a2  a2  a2
i2            
x2   1   1   1
y2   2   2   2
z2   3   3   3
   0  1  2
0  1  1  1
c3  a3  a3  a3
i3            
x3   1   1   1
c4  a4  a4  a4
i4            
x4   1   1   1
y4   2   2   2
z4   3   3   3
c5  a5  a5  a5  b5  b5  b5  c5  c5  c5
i5                                    
x5   1   1   1   2   2   2   3   3   3
c6  a6  a6  a6  b6  b6  b6  c6  c6  c6
i6                                    
x6   1   1   1   2   2   2   3   3   3
y6   4   4   4   5   5   5   6   6   6
z6   7   7   7   8   8   8   9   9   9


In [43]:
print(reshape_fns.tile(v1, 3, axis=0))
print(reshape_fns.tile(a1, 3, axis=0))
print(reshape_fns.tile(a2, 3, axis=0))
print(reshape_fns.tile(a3, 3, axis=0))
print(reshape_fns.tile(a4, 3, axis=0))
print(reshape_fns.tile(a5, 3, axis=0))
print(reshape_fns.tile(sr_none, 3, axis=0))
print(reshape_fns.tile(sr1, 3, axis=0))
print(reshape_fns.tile(sr2, 3, axis=0))
print(reshape_fns.tile(df_none, 3, axis=0))
print(reshape_fns.tile(df1, 3, axis=0))
print(reshape_fns.tile(df2, 3, axis=0))
print(reshape_fns.tile(df3, 3, axis=0))
print(reshape_fns.tile(df4, 3, axis=0))

[0 0 0]
[1 1 1]
[1 2 3 1 2 3 1 2 3]
[[1 2 3]
 [1 2 3]
 [1 2 3]]
[[1]
 [2]
 [3]
 [1]
 [2]
 [3]
 [1]
 [2]
 [3]]
[[1 2 3]
 [4 5 6]
 [7 8 9]
 [1 2 3]
 [4 5 6]
 [7 8 9]
 [1 2 3]
 [4 5 6]
 [7 8 9]]
0    1
1    1
2    1
dtype: int64
i1
x1    1
x1    1
x1    1
Name: a1, dtype: int64
i2
x2    1
y2    2
z2    3
x2    1
y2    2
z2    3
x2    1
y2    2
z2    3
Name: a2, dtype: int64
   0
0  1
1  1
2  1
c3  a3
i3    
x3   1
x3   1
x3   1
c4  a4
i4    
x4   1
y4   2
z4   3
x4   1
y4   2
z4   3
x4   1
y4   2
z4   3
c5  a5  b5  c5
i5            
x5   1   2   3
x5   1   2   3
x5   1   2   3
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
x6   1   2   3
y6   4   5   6
z6   7   8   9
x6   1   2   3
y6   4   5   6
z6   7   8   9


In [44]:
print(reshape_fns.tile(v1, 3, axis=1))
print(reshape_fns.tile(a1, 3, axis=1))
print(reshape_fns.tile(a2, 3, axis=1))
print(reshape_fns.tile(a3, 3, axis=1))
print(reshape_fns.tile(a4, 3, axis=1))
print(reshape_fns.tile(a5, 3, axis=1))
print(reshape_fns.tile(sr_none, 3, axis=1))
print(reshape_fns.tile(sr1, 3, axis=1))
print(reshape_fns.tile(sr2, 3, axis=1))
print(reshape_fns.tile(df_none, 3, axis=1))
print(reshape_fns.tile(df1, 3, axis=1))
print(reshape_fns.tile(df2, 3, axis=1))
print(reshape_fns.tile(df3, 3, axis=1))
print(reshape_fns.tile(df4, 3, axis=1))

[[0 0 0]]
[[1 1 1]]
[[1 1 1]
 [2 2 2]
 [3 3 3]]
[[1 2 3 1 2 3 1 2 3]]
[[1 1 1]
 [2 2 2]
 [3 3 3]]
[[1 2 3 1 2 3 1 2 3]
 [4 5 6 4 5 6 4 5 6]
 [7 8 9 7 8 9 7 8 9]]
   0  1  2
0  1  1  1
    a1  a1  a1
i1            
x1   1   1   1
    a2  a2  a2
i2            
x2   1   1   1
y2   2   2   2
z2   3   3   3
   0  1  2
0  1  1  1
c3  a3  a3  a3
i3            
x3   1   1   1
c4  a4  a4  a4
i4            
x4   1   1   1
y4   2   2   2
z4   3   3   3
c5  a5  b5  c5  a5  b5  c5  a5  b5  c5
i5                                    
x5   1   2   3   1   2   3   1   2   3
c6  a6  b6  c6  a6  b6  c6  a6  b6  c6
i6                                    
x6   1   2   3   1   2   3   1   2   3
y6   4   5   6   4   5   6   4   5   6
z6   7   8   9   7   8   9   7   8   9


In [45]:
# Change broadcasting rules globally
vbt.settings.broadcasting['index_from'] = 'stack' # default is 'strict'
vbt.settings.broadcasting['columns_from'] = 'stack'

print(vbt.settings.broadcasting)

Config({
    "align_index": false,
    "align_columns": true,
    "index_from": "stack",
    "columns_from": "stack",
    "ignore_sr_names": true,
    "drop_duplicates": true,
    "keep": "last",
    "drop_redundant": true,
    "ignore_default": true
})


In [46]:
# Broadcasting arrays
args = [
    ('v1', v1),
    ('a1', a1),
    ('a2', a2),
    ('a3', a3),
    ('a4', a4),
    ('a5', a5)
]
arg_combs = list(itertools.combinations_with_replacement(args, 2))

for (n1, arg1), (n2, arg2) in arg_combs:
    print(arg1)
    print(arg2)
    print("================")
    arg1, arg2 = reshape_fns.broadcast(arg1, arg2)
    print(arg1)
    print(arg2)
    print()

0
0
0
0

0
[1]
[0]
[1]

0
[1 2 3]
[0 0 0]
[1 2 3]

0
[[1 2 3]]
[[0 0 0]]
[[1 2 3]]

0
[[1]
 [2]
 [3]]
[[0]
 [0]
 [0]]
[[1]
 [2]
 [3]]

0
[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[0 0 0]
 [0 0 0]
 [0 0 0]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]

[1]
[1]
[1]
[1]

[1]
[1 2 3]
[1 1 1]
[1 2 3]

[1]
[[1 2 3]]
[[1 1 1]]
[[1 2 3]]

[1]
[[1]
 [2]
 [3]]
[[1]
 [1]
 [1]]
[[1]
 [2]
 [3]]

[1]
[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 1 1]
 [1 1 1]
 [1 1 1]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]

[1 2 3]
[1 2 3]
[1 2 3]
[1 2 3]

[1 2 3]
[[1 2 3]]
[[1 2 3]]
[[1 2 3]]

[1 2 3]
[[1]
 [2]
 [3]]
[[1 2 3]
 [1 2 3]
 [1 2 3]]
[[1 1 1]
 [2 2 2]
 [3 3 3]]

[1 2 3]
[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 2 3]
 [1 2 3]
 [1 2 3]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]

[[1 2 3]]
[[1 2 3]]
[[1 2 3]]
[[1 2 3]]

[[1 2 3]]
[[1]
 [2]
 [3]]
[[1 2 3]
 [1 2 3]
 [1 2 3]]
[[1 1 1]
 [2 2 2]
 [3 3 3]]

[[1 2 3]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[1 2 3]
 [1 2 3]
 [1 2 3]]
[[1 2 3]
 [4 5 6]
 [7 8 9]]

[[1]
 [2]
 [3]]
[[1]
 [2]
 [3]]
[[1]
 [2]
 [3]]
[[1]
 [2]
 [3]]

[[1]
 [2]
 [3]]
[[1 2 3]


In [47]:
# Broadcasting series
args = [
    ('sr_none', sr_none),
    ('sr1', sr1),
    ('sr2', sr2)
]
arg_combs = list(itertools.combinations_with_replacement(args, 2))

for (n1, arg1), (n2, arg2) in arg_combs:
    print(n1 + '+' + n2)
    print(arg1)
    print(arg2)
    print("================")
    arg1, arg2 = reshape_fns.broadcast(arg1, arg2)
    print(arg1)
    print(arg2)
    print()

sr_none+sr_none
0    1
dtype: int64
0    1
dtype: int64
0    1
dtype: int64
0    1
dtype: int64

sr_none+sr1
0    1
dtype: int64
i1
x1    1
Name: a1, dtype: int64
i1
x1    1
dtype: int64
i1
x1    1
dtype: int64

sr_none+sr2
0    1
dtype: int64
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64
i2
x2    1
y2    1
z2    1
dtype: int64
i2
x2    1
y2    2
z2    3
dtype: int64

sr1+sr1
i1
x1    1
Name: a1, dtype: int64
i1
x1    1
Name: a1, dtype: int64
i1
x1    1
Name: a1, dtype: int64
i1
x1    1
Name: a1, dtype: int64

sr1+sr2
i1
x1    1
Name: a1, dtype: int64
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64
i1  i2
x1  x2    1
    y2    1
    z2    1
dtype: int64
i1  i2
x1  x2    1
    y2    2
    z2    3
dtype: int64

sr2+sr2
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64



In [48]:
# Broadcasting arrays and series
a_args = [
    ('v1', v1),
    ('a1', a1),
    ('a2', a2),
    ('a3', a3),
    ('a4', a4),
    ('a5', a5)
]
sr_args = [
    ('sr_none', sr_none),
    ('sr1', sr1),
    ('sr2', sr2)
]
arg_combs = list(itertools.product(a_args, sr_args))

for (n1, arg1), (n2, arg2) in arg_combs:
    print(n1 + '+' + n2)
    print(arg1)
    print(arg2)
    print("================")
    arg1, arg2 = reshape_fns.broadcast(arg1, arg2)
    print(arg1)
    print(arg2)
    print()

v1+sr_none
0
0    1
dtype: int64
0    0
dtype: int64
0    1
dtype: int64

v1+sr1
0
i1
x1    1
Name: a1, dtype: int64
i1
x1    0
Name: a1, dtype: int64
i1
x1    1
Name: a1, dtype: int64

v1+sr2
0
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64
i2
x2    0
y2    0
z2    0
Name: a2, dtype: int64
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64

a1+sr_none
[1]
0    1
dtype: int64
0    1
dtype: int64
0    1
dtype: int64

a1+sr1
[1]
i1
x1    1
Name: a1, dtype: int64
i1
x1    1
Name: a1, dtype: int64
i1
x1    1
Name: a1, dtype: int64

a1+sr2
[1]
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64
i2
x2    1
y2    1
z2    1
Name: a2, dtype: int64
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64

a2+sr_none
[1 2 3]
0    1
dtype: int64
0    1
1    2
2    3
dtype: int64
0    1
1    1
2    1
dtype: int64

a2+sr1
[1 2 3]
i1
x1    1
Name: a1, dtype: int64
i1
x1    1
x1    2
x1    3
Name: a1, dtype: int64
i1
x1    1
x1    1
x1    1
Name: a1, dtype: int64

a2+sr2
[1 2 3]
i2
x2    1
y2    2
z2    3
Na

In [49]:
# Broadcasting dataframes
args = [
    ('df_none', df_none),
    ('df1', df1),
    ('df2', df2),
    ('df3', df3),
    ('df4', df4)
]
arg_combs = list(itertools.combinations_with_replacement(args, 2))

for (n1, arg1), (n2, arg2) in arg_combs:
    print(n1 + '+' + n2)
    print(arg1)
    print(arg2)
    print("================")
    arg1, arg2 = reshape_fns.broadcast(arg1, arg2)
    print(arg1)
    print(arg2)
    print()

df_none+df_none
   0
0  1
   0
0  1
   0
0  1
   0
0  1

df_none+df1
   0
0  1
c3  a3
i3    
x3   1
c3  a3
i3    
x3   1
c3  a3
i3    
x3   1

df_none+df2
   0
0  1
c4  a4
i4    
x4   1
y4   2
z4   3
c4  a4
i4    
x4   1
y4   1
z4   1
c4  a4
i4    
x4   1
y4   2
z4   3

df_none+df3
   0
0  1
c5  a5  b5  c5
i5            
x5   1   2   3
c5  a5  b5  c5
i5            
x5   1   1   1
c5  a5  b5  c5
i5            
x5   1   2   3

df_none+df4
   0
0  1
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
c6  a6  b6  c6
i6            
x6   1   1   1
y6   1   1   1
z6   1   1   1
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9

df1+df1
c3  a3
i3    
x3   1
c3  a3
i3    
x3   1
c3  a3
i3    
x3   1
c3  a3
i3    
x3   1

df1+df2
c3  a3
i3    
x3   1
c4  a4
i4    
x4   1
y4   2
z4   3
c3    a3
c4    a4
i3 i4   
x3 x4  1
   y4  1
   z4  1
c3    a3
c4    a4
i3 i4   
x3 x4  1
   y4  2
   z4  3

df1+df3
c3  a3
i3    
x3   1
c5  a5  b5  c5
i5           

In [50]:
# Broadcasting arrays and dataframes
a_args = [
    ('v1', v1),
    ('a1', a1),
    ('a2', a2),
    ('a3', a3),
    ('a4', a4),
    ('a5', a5)
]
sr_args = [
    ('df_none', df_none),
    ('df1', df1),
    ('df2', df2),
    ('df3', df3),
    ('df4', df4)
]
arg_combs = list(itertools.product(a_args, sr_args))

for (n1, arg1), (n2, arg2) in arg_combs:
    print(n1 + '+' + n2)
    print(arg1)
    print(arg2)
    print("================")
    arg1, arg2 = reshape_fns.broadcast(arg1, arg2)
    print(arg1)
    print(arg2)
    print()

v1+df_none
0
   0
0  1
   0
0  0
   0
0  1

v1+df1
0
c3  a3
i3    
x3   1
c3  a3
i3    
x3   0
c3  a3
i3    
x3   1

v1+df2
0
c4  a4
i4    
x4   1
y4   2
z4   3
c4  a4
i4    
x4   0
y4   0
z4   0
c4  a4
i4    
x4   1
y4   2
z4   3

v1+df3
0
c5  a5  b5  c5
i5            
x5   1   2   3
c5  a5  b5  c5
i5            
x5   0   0   0
c5  a5  b5  c5
i5            
x5   1   2   3

v1+df4
0
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
c6  a6  b6  c6
i6            
x6   0   0   0
y6   0   0   0
z6   0   0   0
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9

a1+df_none
[1]
   0
0  1
   0
0  1
   0
0  1

a1+df1
[1]
c3  a3
i3    
x3   1
c3  a3
i3    
x3   1
c3  a3
i3    
x3   1

a1+df2
[1]
c4  a4
i4    
x4   1
y4   2
z4   3
c4  a4
i4    
x4   1
y4   1
z4   1
c4  a4
i4    
x4   1
y4   2
z4   3

a1+df3
[1]
c5  a5  b5  c5
i5            
x5   1   2   3
c5  a5  b5  c5
i5            
x5   1   1   1
c5  a5  b5  c5
i5            
x5   1   2   3

a1+

In [51]:
# Broadcasting series and dataframes
a_args = [
    ('sr_none', sr_none),
    ('sr1', sr1),
    ('sr2', sr2)
]
sr_args = [
    ('df_none', df_none),
    ('df1', df1),
    ('df2', df2),
    ('df3', df3),
    ('df4', df4)
]
arg_combs = list(itertools.product(a_args, sr_args))

for (n1, arg1), (n2, arg2) in arg_combs:
    print(n1 + '+' + n2)
    print(arg1)
    print(arg2)
    print("================")
    arg1, arg2 = reshape_fns.broadcast(arg1, arg2)
    print(arg1)
    print(arg2)
    print()

sr_none+df_none
0    1
dtype: int64
   0
0  1
   0
0  1
   0
0  1

sr_none+df1
0    1
dtype: int64
c3  a3
i3    
x3   1
c3  a3
i3    
x3   1
c3  a3
i3    
x3   1

sr_none+df2
0    1
dtype: int64
c4  a4
i4    
x4   1
y4   2
z4   3
c4  a4
i4    
x4   1
y4   1
z4   1
c4  a4
i4    
x4   1
y4   2
z4   3

sr_none+df3
0    1
dtype: int64
c5  a5  b5  c5
i5            
x5   1   2   3
c5  a5  b5  c5
i5            
x5   1   1   1
c5  a5  b5  c5
i5            
x5   1   2   3

sr_none+df4
0    1
dtype: int64
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
c6  a6  b6  c6
i6            
x6   1   1   1
y6   1   1   1
z6   1   1   1
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9

sr1+df_none
i1
x1    1
Name: a1, dtype: int64
   0
0  1
    0
i1   
x1  1
    0
i1   
x1  1

sr1+df1
i1
x1    1
Name: a1, dtype: int64
c3  a3
i3    
x3   1
c3     a3
i1 i3    
x1 x3   1
c3     a3
i1 i3    
x1 x3   1

sr1+df2
i1
x1    1
Name: a1, dtype: int64
c4  a4
i4    


In [52]:
[np.broadcast_to(x, (3, 3)) for x in (0, a1, a2, sr_none, sr1, sr2)]

[array([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]),
 array([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]),
 array([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]]),
 array([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]),
 array([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]),
 array([[1, 2, 3],
        [1, 2, 3],
        [1, 2, 3]])]

In [53]:
# Broadcasting all at once
for i in reshape_fns.broadcast(
    0, a1, a2, sr_none, sr1, sr2,
    to_shape=(3, 3),
    index_from='stack',
    columns_from='stack'
):
    print(i)

       0  1  2
i1 i2         
x1 x2  0  0  0
   y2  0  0  0
   z2  0  0  0
       0  1  2
i1 i2         
x1 x2  1  1  1
   y2  1  1  1
   z2  1  1  1
       0  1  2
i1 i2         
x1 x2  1  2  3
   y2  1  2  3
   z2  1  2  3
       0  1  2
i1 i2         
x1 x2  1  1  1
   y2  1  1  1
   z2  1  1  1
       0  1  2
i1 i2         
x1 x2  1  1  1
   y2  1  1  1
   z2  1  1  1
       0  1  2
i1 i2         
x1 x2  1  1  1
   y2  2  2  2
   z2  3  3  3


In [54]:
# Broadcasting all at once
for i in reshape_fns.broadcast(
    v1, a1, a2, a3, a4, a5, sr_none, sr1, sr2, df_none, df1, df2, df3, df4,
    index_from='stack',
    columns_from='stack'
):
    print(i)

c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x1 x2 x3 x4 x5 x6  0  0  0
   y2 x3 y4 x5 y6  0  0  0
   z2 x3 z4 x5 z6  0  0  0
c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x1 x2 x3 x4 x5 x6  1  1  1
   y2 x3 y4 x5 y6  1  1  1
   z2 x3 z4 x5 z6  1  1  1
c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x1 x2 x3 x4 x5 x6  1  2  3
   y2 x3 y4 x5 y6  1  2  3
   z2 x3 z4 x5 z6  1  2  3
c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x1 x2 x3 x4 x5 x6  1  2  3
   y2 x3 y4 x5 y6  1  2  3
   z2 x3 z4 x5 z6  1  2  3
c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x

In [55]:
for i in reshape_fns.broadcast(
    v1, a1, a2, a3, a4, a5, sr_none, sr1, sr2, df_none, df1, df2, df3, df4,
    index_from=None, # use as-is
    columns_from=None
):
    print(i)

c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x1 x2 x3 x4 x5 x6  0  0  0
   y2 x3 y4 x5 y6  0  0  0
   z2 x3 z4 x5 z6  0  0  0
c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x1 x2 x3 x4 x5 x6  1  1  1
   y2 x3 y4 x5 y6  1  1  1
   z2 x3 z4 x5 z6  1  1  1
c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x1 x2 x3 x4 x5 x6  1  2  3
   y2 x3 y4 x5 y6  1  2  3
   z2 x3 z4 x5 z6  1  2  3
c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x1 x2 x3 x4 x5 x6  1  2  3
   y2 x3 y4 x5 y6  1  2  3
   z2 x3 z4 x5 z6  1  2  3
c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x

In [56]:
for i in reshape_fns.broadcast(
    v1, a1, a2, a3, a4, a5, sr_none, sr1, sr2, df_none, df1, df2, df3, df4,
    index_from=-1, # take index from the last dataframe
    columns_from=-1
):
    print(i)

c6  a6  b6  c6
i6            
x6   0   0   0
y6   0   0   0
z6   0   0   0
c6  a6  b6  c6
i6            
x6   1   1   1
y6   1   1   1
z6   1   1   1
c6  a6  b6  c6
i6            
x6   1   2   3
y6   1   2   3
z6   1   2   3
c6  a6  b6  c6
i6            
x6   1   2   3
y6   1   2   3
z6   1   2   3
c6  a6  b6  c6
i6            
x6   1   1   1
y6   2   2   2
z6   3   3   3
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
c6  a6  b6  c6
i6            
x6   1   1   1
y6   1   1   1
z6   1   1   1
c6  a6  b6  c6
i6            
x6   1   1   1
y6   1   1   1
z6   1   1   1
c6  a6  b6  c6
i6            
x6   1   1   1
y6   2   2   2
z6   3   3   3
c6  a6  b6  c6
i6            
x6   1   1   1
y6   1   1   1
z6   1   1   1
c6  a6  b6  c6
i6            
x6   1   1   1
y6   1   1   1
z6   1   1   1
c6  a6  b6  c6
i6            
x6   1   1   1
y6   2   2   2
z6   3   3   3
c6  a6  b6  c6
i6            
x6   1   2   3
y6   1   2   3
z6   1   2   3
c6  a6  b6  c6
i6        

In [57]:
for i in reshape_fns.broadcast(
    v1, a1, a2, a3, a4, a5, sr_none, sr1, sr2, df_none, df1, df2, df3, df4,
    index_from=multi_i, # specify manually
    columns_from=multi_c
):
    print(i)

c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  0  0  0
y7 y8  0  0  0
z7 z8  0  0  0
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  1  1
y7 y8  1  1  1
z7 z8  1  1  1
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  2  3
y7 y8  1  2  3
z7 z8  1  2  3
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  2  3
y7 y8  1  2  3
z7 z8  1  2  3
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  1  1
y7 y8  2  2  2
z7 z8  3  3  3
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  2  3
y7 y8  4  5  6
z7 z8  7  8  9
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  1  1
y7 y8  1  1  1
z7 z8  1  1  1
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  1  1
y7 y8  1  1  1
z7 z8  1  1  1
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  1  1
y7 y8  2  2  2
z7 z8  3  3  3
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  1  1
y7 y8  1  1  1
z7 z8  1  1  1
c7    a7 b7 c7
c8    a8 b8 c8
i7 i8         
x7 x8  1  1  1
y7 y8  1  1  1
z7 z8  1  1  1
c7    a7 b

In [58]:
# Do not clean columns
vbt.settings.broadcasting['drop_duplicates'] = False
vbt.settings.broadcasting['drop_redundant'] = False
vbt.settings.broadcasting['ignore_sr_names'] = False

for i in reshape_fns.broadcast(
    v1, a1, a2, a3, a4, a5, sr_none, sr1, sr2, df_none, df1, df2, df3, df4,
    index_from='stack', # stack but do not clean
    columns_from='stack'
):
    print(i)
    
vbt.settings.broadcasting.reset()

                  a1      
                  a2      
c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x1 x2 x3 x4 x5 x6  0  0  0
   y2 x3 y4 x5 y6  0  0  0
   z2 x3 z4 x5 z6  0  0  0
                  a1      
                  a2      
c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x1 x2 x3 x4 x5 x6  1  1  1
   y2 x3 y4 x5 y6  1  1  1
   z2 x3 z4 x5 z6  1  1  1
                  a1      
                  a2      
c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x1 x2 x3 x4 x5 x6  1  2  3
   y2 x3 y4 x5 y6  1  2  3
   z2 x3 z4 x5 z6  1  2  3
                  a1      
                  a2      
c3                a3      
c4                a4      
c5                a5 b5 c5
c6                a6 b6 c6
i1 i2 i3 i4 i5 i6         
x

In [59]:
big_a = np.empty((1000, 1000))

In [60]:
print(reshape_fns.broadcast(np.empty((1,)), big_a)[0].flags)
%timeit reshape_fns.broadcast(np.empty((1,)), big_a)

print(reshape_fns.broadcast(np.empty((1,)), big_a, require_kwargs={'requirements': 'W'})[0].flags)
%timeit reshape_fns.broadcast(np.empty((1,)), big_a, require_kwargs={'requirements': 'W'})

print(reshape_fns.broadcast(np.empty((1,)), big_a, require_kwargs={'requirements': 'C'})[0].flags)
%timeit reshape_fns.broadcast(np.empty((1,)), big_a, require_kwargs={'requirements': 'C'})

print(reshape_fns.broadcast(np.empty((1,)), big_a, require_kwargs={'requirements': 'F'})[0].flags)
%timeit reshape_fns.broadcast(np.empty((1,)), big_a, require_kwargs={'requirements': 'F'})

  C_CONTIGUOUS : False
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : False
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

19.1 µs ± 471 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

596 µs ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

181 µs ± 11.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
  C_CONTIGUOUS : False
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

1.26 ms ± 35.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [61]:
print(reshape_fns.broadcast(v1, df4, to_pd=False))
print(reshape_fns.broadcast(v1, df4, to_pd=True))

(array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]]), array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 9]]))
(c6  a6  b6  c6
i6            
x6   0   0   0
y6   0   0   0
z6   0   0   0, c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9)


In [62]:
# One-side broadcasting, default behaviour is copying index/columns from the second argument
print(reshape_fns.broadcast_to(sr1, sr1))
print(reshape_fns.broadcast_to(sr1, sr2))
print(reshape_fns.broadcast_to(sr1, df1))
print(reshape_fns.broadcast_to(sr1, df2))
print(reshape_fns.broadcast_to(sr1, df3))
print(reshape_fns.broadcast_to(sr1, df4))

i1
x1    1
Name: a1, dtype: int64
i2
x2    1
y2    1
z2    1
Name: a2, dtype: int64
c3  a3
i3    
x3   1
c4  a4
i4    
x4   1
y4   1
z4   1
c5  a5  b5  c5
i5            
x5   1   1   1
c6  a6  b6  c6
i6            
x6   1   1   1
y6   1   1   1
z6   1   1   1


In [63]:
# Broadcasting first element to be an array out of the second argument
print(reshape_fns.broadcast_to_array_of(0.1, v1))
print(reshape_fns.broadcast_to_array_of([0.1], v1))
print(reshape_fns.broadcast_to_array_of([0.1, 0.2], v1))

[0.1]
[0.1]
[0.1 0.2]


In [64]:
print(reshape_fns.broadcast_to_array_of(0.1, sr2))
print(reshape_fns.broadcast_to_array_of([0.1], sr2))
print(reshape_fns.broadcast_to_array_of([0.1, 0.2], sr2))
print(reshape_fns.broadcast_to_array_of([[0.1, 0.2, 0.3], [0.4, 0.5, 0.6]], sr2))

[[0.1 0.1 0.1]]
[[0.1 0.1 0.1]]
[[0.1 0.1 0.1]
 [0.2 0.2 0.2]]
[[0.1 0.2 0.3]
 [0.4 0.5 0.6]]


In [65]:
print(reshape_fns.broadcast_to_array_of(0.1, df2))
print(reshape_fns.broadcast_to_array_of([0.1], df2))
print(reshape_fns.broadcast_to_array_of([0.1, 0.2], df2))
print(reshape_fns.broadcast_to_array_of([[[0.1], [0.2], [0.3]], [[0.4], [0.5], [0.6]]], df2))

[[[0.1]
  [0.1]
  [0.1]]]
[[[0.1]
  [0.1]
  [0.1]]]
[[[0.1]
  [0.1]
  [0.1]]

 [[0.2]
  [0.2]
  [0.2]]]
[[[0.1]
  [0.2]
  [0.3]]

 [[0.4]
  [0.5]
  [0.6]]]


In [66]:
print(reshape_fns.broadcast_to_array_of(0.1, np.empty((2, 2, 2)))) # works even for ndim > 2

[[[[0.1 0.1]
   [0.1 0.1]]

  [[0.1 0.1]
   [0.1 0.1]]]]


In [67]:
print(reshape_fns.broadcast_to_axis_of(10, np.empty((2,)), 0))
print(reshape_fns.broadcast_to_axis_of(10, np.empty((2,)), 1))
print(reshape_fns.broadcast_to_axis_of(10, np.empty((2, 3)), 0))
print(reshape_fns.broadcast_to_axis_of(10, np.empty((2, 3)), 1))
print(reshape_fns.broadcast_to_axis_of(10, np.empty((2, 3)), 2))

[10 10]
10
[10 10]
[10 10 10]
10


In [68]:
i = pd.MultiIndex.from_arrays([[1, 1, 2, 2], [3, 4, 3, 4], ['a', 'b', 'c', 'd']])
sr = pd.Series([1, 2, 3, 4], index=i)
print(reshape_fns.unstack_to_array(sr))

[[[ 1. nan nan nan]
  [nan  2. nan nan]]

 [[nan nan  3. nan]
  [nan nan nan  4.]]]


In [69]:
print(reshape_fns.make_symmetric(sr1))
print(reshape_fns.make_symmetric(sr2))
print(reshape_fns.make_symmetric(df1))
print(reshape_fns.make_symmetric(df2))
print(reshape_fns.make_symmetric(df3))
print(reshape_fns.make_symmetric(df4))
print(reshape_fns.make_symmetric(df5))
print(reshape_fns.make_symmetric(pd.Series([1, 2, 3], name='yo'), sort=False))

('i1', None)   a1   x1
(i1, None)            
a1            NaN  1.0
x1            1.0  NaN
('i2', None)   a2   x2   y2   z2
(i2, None)                      
a2            NaN  1.0  2.0  3.0
x2            1.0  NaN  NaN  NaN
y2            2.0  NaN  NaN  NaN
z2            3.0  NaN  NaN  NaN
('i3', 'c3')   a3   x3
(i3, c3)              
a3            NaN  1.0
x3            1.0  NaN
('i4', 'c4')   a4   x4   y4   z4
(i4, c4)                        
a4            NaN  1.0  2.0  3.0
x4            1.0  NaN  NaN  NaN
y4            2.0  NaN  NaN  NaN
z4            3.0  NaN  NaN  NaN
('i5', 'c5')   a5   b5   c5   x5
(i5, c5)                        
a5            NaN  NaN  NaN  1.0
b5            NaN  NaN  NaN  2.0
c5            NaN  NaN  NaN  3.0
x5            1.0  2.0  3.0  NaN
('i6', 'c6')   a6   b6   c6   x6   y6   z6
(i6, c6)                                  
a6            NaN  NaN  NaN  1.0  4.0  7.0
b6            NaN  NaN  NaN  2.0  5.0  8.0
c6            NaN  NaN  NaN  3.0  6.0  9.0
x6     

In [70]:
print(reshape_fns.unstack_to_df(df5.iloc[0]))
print(reshape_fns.unstack_to_df(sr, index_levels=0, column_levels=1))
print(reshape_fns.unstack_to_df(sr, index_levels=(0, 1), column_levels=2))
print(reshape_fns.unstack_to_df(sr, index_levels=0, column_levels=1, symmetric=True).columns)

c8   a8   b8   c8
c7               
a7  1.0  NaN  NaN
b7  NaN  2.0  NaN
c7  NaN  NaN  3.0
     3    4
1  1.0  2.0
2  3.0  4.0
       a    b    c    d
1 3  1.0  NaN  NaN  NaN
  4  NaN  2.0  NaN  NaN
2 3  NaN  NaN  3.0  NaN
  4  NaN  NaN  NaN  4.0
Index([1, 2, 3, 4], dtype='int64')


## indexing

In [71]:
PandasIndexer = indexing.PandasIndexer
ParamIndexer = indexing.build_param_indexer(['param1', 'param2', 'tuple'])

class H(PandasIndexer, ParamIndexer):
    def __init__(self, a, param1_mapper, param2_mapper, tuple_mapper):
        self.a = a
        
        self._param1_mapper = param1_mapper
        self._param2_mapper = param2_mapper
        self._tuple_mapper = tuple_mapper
        
        PandasIndexer.__init__(self, my_kw='PandasIndexer')
        ParamIndexer.__init__(self, [param1_mapper, param2_mapper, tuple_mapper], my_kw='ParamIndexer')
        
    def indexing_func(self, pd_indexing_func, my_kw=None): 
        # As soon as you call iloc etc., performs it on each dataframe and mapper and returns a new class instance
        print(my_kw)
        param1_mapper = indexing.indexing_on_mapper(self._param1_mapper, self.a, pd_indexing_func)
        param2_mapper = indexing.indexing_on_mapper(self._param2_mapper, self.a, pd_indexing_func)
        tuple_mapper = indexing.indexing_on_mapper(self._tuple_mapper, self.a, pd_indexing_func)
        return H(pd_indexing_func(self.a), param1_mapper, param2_mapper, tuple_mapper)
        
    @classmethod
    def run(cls, a, params1, params2, level_names=('p1', 'p2')):
        a = reshape_fns.to_2d(a)
        # Build column hierarchy
        params1_idx = pd.Index(params1, name=level_names[0])
        params2_idx = pd.Index(params2, name=level_names[1])
        params_idx = index_fns.stack_indexes((params1_idx, params2_idx))
        new_columns = index_fns.combine_indexes((params_idx, a.columns))
        
        # Build mappers
        param1_mapper = np.repeat(params1, len(a.columns))
        param1_mapper = pd.Series(param1_mapper, index=new_columns, name=params1_idx.name)
        
        param2_mapper = np.repeat(params2, len(a.columns))
        param2_mapper = pd.Series(param2_mapper, index=new_columns, name=params2_idx.name)
        
        tuple_mapper = list(zip(*list(map(lambda x: x.values, [param1_mapper, param2_mapper]))))
        tuple_mapper = pd.Series(tuple_mapper, index=new_columns, name=(params1_idx.name, params2_idx.name))
        
        # Tile a to match the length of new_columns
        a = array_wrapper.ArrayWrapper(a.index, new_columns, 2).wrap(reshape_fns.tile(a.values, 4, axis=1))
        return cls(a, param1_mapper, param2_mapper, tuple_mapper)
        

# Similate an indicator with two params
h = H.run(df4, [0.1, 0.1, 0.2, 0.2], [0.3, 0.4, 0.5, 0.6])

print(df4)
print(h.a)
print(h._param1_mapper)
print(h._param2_mapper)
print(h._tuple_mapper)

c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9
p1 0.1                 0.2                
p2 0.3       0.4       0.5       0.6      
c6  a6 b6 c6  a6 b6 c6  a6 b6 c6  a6 b6 c6
i6                                        
x6   1  2  3   1  2  3   1  2  3   1  2  3
y6   4  5  6   4  5  6   4  5  6   4  5  6
z6   7  8  9   7  8  9   7  8  9   7  8  9
p1   p2   c6
0.1  0.3  a6    0.1
          b6    0.1
          c6    0.1
     0.4  a6    0.1
          b6    0.1
          c6    0.1
0.2  0.5  a6    0.2
          b6    0.2
          c6    0.2
     0.6  a6    0.2
          b6    0.2
          c6    0.2
Name: p1, dtype: float64
p1   p2   c6
0.1  0.3  a6    0.3
          b6    0.3
          c6    0.3
     0.4  a6    0.4
          b6    0.4
          c6    0.4
0.2  0.5  a6    0.5
          b6    0.5
          c6    0.5
     0.6  a6    0.6
          b6    0.6
          c6    0.6
Name: p2, dtype: float64
p1   p2   c6
0.1  0.3  a6    (0.1, 0.3)
          b6    (0.1, 0.3)
 

In [72]:
# Indexing operations are delegated to the underlying dataframes
print(h[(0.1, 0.3, 'a6')].a)
print(h.loc[:, (0.1, 0.3, 'a6'):(0.1, 0.3, 'c6')].a)
print(h.iloc[-2:, -2:].a)
print(h.xs((0.1, 0.3), level=('p1', 'p2'), axis=1).a.columns)

PandasIndexer
i6
x6    1
y6    4
z6    7
Name: (0.1, 0.3, a6), dtype: int64
PandasIndexer
p1 0.1      
p2 0.3      
c6  a6 b6 c6
i6          
x6   1  2  3
y6   4  5  6
z6   7  8  9
PandasIndexer
p1 0.2   
p2 0.6   
c6  b6 c6
i6       
y6   5  6
z6   8  9
PandasIndexer
Index(['a6', 'b6', 'c6'], dtype='object', name='c6')


In [73]:
print(h.param1_loc[0.1].a.columns)
print(h.param1_loc[0.1:0.1].a)
print(h.param1_loc[[0.1, 0.1]].a)

ParamIndexer
MultiIndex([(0.1, 0.3, 'a6'),
            (0.1, 0.3, 'b6'),
            (0.1, 0.3, 'c6'),
            (0.1, 0.4, 'a6'),
            (0.1, 0.4, 'b6'),
            (0.1, 0.4, 'c6')],
           names=['p1', 'p2', 'c6'])
ParamIndexer
p1 0.1                
p2 0.3       0.4      
c6  a6 b6 c6  a6 b6 c6
i6                    
x6   1  2  3   1  2  3
y6   4  5  6   4  5  6
z6   7  8  9   7  8  9
ParamIndexer
p1 0.1                                    
p2 0.3       0.4       0.3       0.4      
c6  a6 b6 c6  a6 b6 c6  a6 b6 c6  a6 b6 c6
i6                                        
x6   1  2  3   1  2  3   1  2  3   1  2  3
y6   4  5  6   4  5  6   4  5  6   4  5  6
z6   7  8  9   7  8  9   7  8  9   7  8  9


In [74]:
print(h.param2_loc[0.3].a)
print(h.param2_loc[0.3:0.3].a)
print(h.param2_loc[[0.3, 0.3]].a.columns)

ParamIndexer
p1 0.1      
p2 0.3      
c6  a6 b6 c6
i6          
x6   1  2  3
y6   4  5  6
z6   7  8  9
ParamIndexer
p1 0.1      
p2 0.3      
c6  a6 b6 c6
i6          
x6   1  2  3
y6   4  5  6
z6   7  8  9
ParamIndexer
MultiIndex([(0.1, 0.3, 'a6'),
            (0.1, 0.3, 'b6'),
            (0.1, 0.3, 'c6'),
            (0.1, 0.3, 'a6'),
            (0.1, 0.3, 'b6'),
            (0.1, 0.3, 'c6')],
           names=['p1', 'p2', 'c6'])


In [75]:
print(h.tuple_loc[(0.1, 0.3)].a)
print(h.tuple_loc[(0.1, 0.3):(0.1, 0.3)].a.columns)
print(h.tuple_loc[[(0.1, 0.3), (0.1, 0.3)]].a.columns)

ParamIndexer
p1 0.1      
p2 0.3      
c6  a6 b6 c6
i6          
x6   1  2  3
y6   4  5  6
z6   7  8  9
ParamIndexer
MultiIndex([(0.1, 0.3, 'a6'),
            (0.1, 0.3, 'b6'),
            (0.1, 0.3, 'c6')],
           names=['p1', 'p2', 'c6'])
ParamIndexer
MultiIndex([(0.1, 0.3, 'a6'),
            (0.1, 0.3, 'b6'),
            (0.1, 0.3, 'c6'),
            (0.1, 0.3, 'a6'),
            (0.1, 0.3, 'b6'),
            (0.1, 0.3, 'c6')],
           names=['p1', 'p2', 'c6'])


## combine_fns

In [76]:
vbt.settings.broadcasting['index_from'] = 'stack'
vbt.settings.broadcasting['columns_from'] = 'stack'

In [77]:
print(combine_fns.apply_and_concat_one(3, lambda i, x, a: x + a[i], sr2.values, [10, 20, 30]))
print(combine_fns.apply_and_concat_one_nb(3, njit(lambda i, x, a: x + a[i]), sr2.values, (10, 20, 30)))

print(combine_fns.apply_and_concat_one(3, lambda i, x, a: x + a[i], df4.values, [10, 20, 30]))
print(combine_fns.apply_and_concat_one_nb(3, njit(lambda i, x, a: x + a[i]), df4.values, (10, 20, 30)))

[[11 21 31]
 [12 22 32]
 [13 23 33]]
[[11 21 31]
 [12 22 32]
 [13 23 33]]
[[11 12 13 21 22 23 31 32 33]
 [14 15 16 24 25 26 34 35 36]
 [17 18 19 27 28 29 37 38 39]]
[[11 12 13 21 22 23 31 32 33]
 [14 15 16 24 25 26 34 35 36]
 [17 18 19 27 28 29 37 38 39]]


In [78]:
print(combine_fns.apply_and_concat_multiple(3, lambda i, x, a: (x, x + a[i]), sr2.values, [10, 20, 30]))
print(combine_fns.apply_and_concat_multiple_nb(3, njit(lambda i, x, a: (x, x + a[i])), sr2.values, (10, 20, 30)))

print(combine_fns.apply_and_concat_multiple(3, lambda i, x, a: (x, x + a[i]), df4.values, [10, 20, 30]))
print(combine_fns.apply_and_concat_multiple_nb(3, njit(lambda i, x, a: (x, x + a[i])), df4.values, (10, 20, 30)))

[array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]]), array([[11, 21, 31],
       [12, 22, 32],
       [13, 23, 33]])]
[array([[1, 1, 1],
       [2, 2, 2],
       [3, 3, 3]]), array([[11, 21, 31],
       [12, 22, 32],
       [13, 23, 33]])]
[array([[1, 2, 3, 1, 2, 3, 1, 2, 3],
       [4, 5, 6, 4, 5, 6, 4, 5, 6],
       [7, 8, 9, 7, 8, 9, 7, 8, 9]]), array([[11, 12, 13, 21, 22, 23, 31, 32, 33],
       [14, 15, 16, 24, 25, 26, 34, 35, 36],
       [17, 18, 19, 27, 28, 29, 37, 38, 39]])]
[array([[1, 2, 3, 1, 2, 3, 1, 2, 3],
       [4, 5, 6, 4, 5, 6, 4, 5, 6],
       [7, 8, 9, 7, 8, 9, 7, 8, 9]]), array([[11, 12, 13, 21, 22, 23, 31, 32, 33],
       [14, 15, 16, 24, 25, 26, 34, 35, 36],
       [17, 18, 19, 27, 28, 29, 37, 38, 39]])]


In [79]:
print(combine_fns.combine_and_concat(sr2.values, (sr2.values*2, sr2.values*3), lambda x, y, a: x + y + a, 100))
print(combine_fns.combine_and_concat_nb(sr2.values, (sr2.values*2, sr2.values*3), njit(lambda x, y, a: x + y + a), 100))

print(combine_fns.combine_and_concat(df4.values, (df4.values*2, df4.values*3), lambda x, y, a: x + y + a, 100))
print(combine_fns.combine_and_concat_nb(df4.values, (df4.values*2, df4.values*3), njit(lambda x, y, a: x + y + a), 100))

[[103 104]
 [106 108]
 [109 112]]
[[103 104]
 [106 108]
 [109 112]]
[[103 106 109 104 108 112]
 [112 115 118 116 120 124]
 [121 124 127 128 132 136]]
[[103 106 109 104 108 112]
 [112 115 118 116 120 124]
 [121 124 127 128 132 136]]


In [80]:
print(combine_fns.combine_multiple((sr2.values, sr2.values*2, sr2.values*3), lambda x, y, a: x + y + a, 100))
print(combine_fns.combine_multiple_nb((sr2.values, sr2.values*2, sr2.values*3), njit(lambda x, y, a: x + y + a), 100))

print(combine_fns.combine_multiple((df4.values, df4.values*2, df4.values*3), lambda x, y, a: x + y + a, 100))
print(combine_fns.combine_multiple_nb((df4.values, df4.values*2, df4.values*3), njit(lambda x, y, a: x + y + a), 100))

[206 212 218]
[206 212 218]
[[206 212 218]
 [224 230 236]
 [242 248 254]]
[[206 212 218]
 [224 230 236]
 [242 248 254]]


## accessors

In [81]:
print(pd.Series.vbt.empty(5, index=np.arange(10, 15), name='a', fill_value=5))
print(pd.DataFrame.vbt.empty((5, 3), index=np.arange(10, 15), columns=['a', 'b', 'c'], fill_value=5))

print(pd.Series.vbt.empty_like(sr2, fill_value=5))
print(pd.DataFrame.vbt.empty_like(df4, fill_value=5))

10    5
11    5
12    5
13    5
14    5
Name: a, dtype: int64
    a  b  c
10  5  5  5
11  5  5  5
12  5  5  5
13  5  5  5
14  5  5  5
i2
x2    5
y2    5
z2    5
Name: a2, dtype: int64
c6  a6  b6  c6
i6            
x6   5   5   5
y6   5   5   5
z6   5   5   5


In [82]:
print(sr1.vbt.is_series())
print(sr1.vbt.is_frame())
print(df1.vbt.is_series())
print(df2.vbt.is_frame())

True
False
False
True


In [83]:
print(sr2.vbt.wrapper.index)
print(sr2.vbt.wrapper.columns)
print(df4.vbt.wrapper.index)
print(df4.vbt.wrapper.columns)

Index(['x2', 'y2', 'z2'], dtype='object', name='i2')
Index(['a2'], dtype='object')
Index(['x6', 'y6', 'z6'], dtype='object', name='i6')
Index(['a6', 'b6', 'c6'], dtype='object', name='c6')


In [84]:
print(df1.vbt.apply_on_index(lambda idx: idx + '_yo', axis=0))
print(df1.vbt.apply_on_index(lambda idx: idx + '_yo', axis=1))
df1_copy = df1.copy()
df1_copy.vbt.apply_on_index(lambda idx: idx + '_yo', axis=0, inplace=True)
print(df1_copy)
df1_copy.vbt.apply_on_index(lambda idx: idx + '_yo', axis=1, inplace=True)
print(df1_copy)

c3     a3
i3       
x3_yo   1
c3  a3_yo
i3       
x3      1
c3     a3
i3       
x3_yo   1
c3     a3_yo
i3          
x3_yo      1


In [85]:
print(sr2.vbt.to_1d_array())
print(sr2.vbt.to_2d_array())

[1 2 3]
[[1]
 [2]
 [3]]


In [86]:
# It will try to return pd.Series
print(sr2.vbt.wrapper.wrap(a2)) # returns sr
print(sr2.vbt.wrapper.wrap(df2.values)) # returns sr
print(sr2.vbt.wrapper.wrap(df2.values, index=df2.index, columns=df2.columns)) # returns sr
print(sr2.vbt.wrapper.wrap(df4.values, columns=df4.columns)) # returns df
print(sr2.vbt.wrapper.wrap(df4.values, index=df4.index, columns=df4.columns)) # returns df

i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64
i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64
i4
x4    1
y4    2
z4    3
Name: a4, dtype: int64
c6  a6  b6  c6
i2            
x2   1   2   3
y2   4   5   6
z2   7   8   9
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9


In [87]:
# It will try to return pd.DataFrame
print(df2.vbt.wrapper.wrap(a2)) # returns df
print(df2.vbt.wrapper.wrap(sr2.values)) # returns df
print(df2.vbt.wrapper.wrap(df4.values, columns=df4.columns)) # returns df
print(df2.vbt.wrapper.wrap(df4.values, index=df4.index, columns=df4.columns)) # returns df

c4  a4
i4    
x4   1
y4   2
z4   3
c4  a4
i4    
x4   1
y4   2
z4   3
c6  a6  b6  c6
i4            
x4   1   2   3
y4   4   5   6
z4   7   8   9
c6  a6  b6  c6
i6            
x6   1   2   3
y6   4   5   6
z6   7   8   9


In [88]:
print(df4.vbt.tile(2, keys=['a', 'b']))
print(df4.vbt.repeat(2, keys=['a', 'b']))

    a        b      
c6 a6 b6 c6 a6 b6 c6
i6                  
x6  1  2  3  1  2  3
y6  4  5  6  4  5  6
z6  7  8  9  7  8  9
c6 a6    b6    c6   
    a  b  a  b  a  b
i6                  
x6  1  1  2  2  3  3
y6  4  4  5  5  6  6
z6  7  7  8  8  9  9


In [89]:
df10 = pd.DataFrame([[1, 2], [4, 5], [7, 8]], columns=multi_c1)
df20 = pd.DataFrame([[1, 2, 3, 4], [4, 5, 6, 7], [7, 8, 9, 10]], columns=multi_c2)

print(df10)
print(df20)
print(df10.vbt.align_to(df20))

c8 a8 b8
0   1  2
1   4  5
2   7  8
c7 a7    c7    
c8 a8 b8 a8  b8
0   1  2  3   4
1   4  5  6   7
2   7  8  9  10
c7 a7    c7   
c8 a8 b8 a8 b8
0   1  2  1  2
1   4  5  4  5
2   7  8  7  8


In [90]:
print(pd.DataFrame.vbt.broadcast(
    sr2,
    10
))
print(sr2.vbt.broadcast(
    10
))
print(sr2.vbt.broadcast_to(
    df2
))

(i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64, i2
x2    10
y2    10
z2    10
Name: a2, dtype: int64)
(i2
x2    1
y2    2
z2    3
Name: a2, dtype: int64, i2
x2    10
y2    10
z2    10
Name: a2, dtype: int64)
c4  a4
i4    
x4   1
y4   2
z4   3


In [91]:
print(sr2.vbt.make_symmetric())
print(df2.vbt.make_symmetric())
print(df3.vbt.make_symmetric())
print(df4.vbt.make_symmetric())

('i2', None)   a2   x2   y2   z2
(i2, None)                      
a2            NaN  1.0  2.0  3.0
x2            1.0  NaN  NaN  NaN
y2            2.0  NaN  NaN  NaN
z2            3.0  NaN  NaN  NaN
('i4', 'c4')   a4   x4   y4   z4
(i4, c4)                        
a4            NaN  1.0  2.0  3.0
x4            1.0  NaN  NaN  NaN
y4            2.0  NaN  NaN  NaN
z4            3.0  NaN  NaN  NaN
('i5', 'c5')   a5   b5   c5   x5
(i5, c5)                        
a5            NaN  NaN  NaN  1.0
b5            NaN  NaN  NaN  2.0
c5            NaN  NaN  NaN  3.0
x5            1.0  2.0  3.0  NaN
('i6', 'c6')   a6   b6   c6   x6   y6   z6
(i6, c6)                                  
a6            NaN  NaN  NaN  1.0  4.0  7.0
b6            NaN  NaN  NaN  2.0  5.0  8.0
c6            NaN  NaN  NaN  3.0  6.0  9.0
x6            1.0  2.0  3.0  NaN  NaN  NaN
y6            4.0  5.0  6.0  NaN  NaN  NaN
z6            7.0  8.0  9.0  NaN  NaN  NaN


In [92]:
print(df5.iloc[:, 0].vbt.unstack_to_array())

[[ 1. nan nan]
 [nan  4. nan]
 [nan nan  7.]]


In [93]:
print(df5.iloc[:, 0].vbt.unstack_to_df())

i8   x8   y8   z8
i7               
x7  1.0  NaN  NaN
y7  NaN  4.0  NaN
z7  NaN  NaN  7.0


In [94]:
print(sr2.vbt.apply(apply_func=lambda x: x ** 2))
print(sr2.vbt.apply(apply_func=lambda x: x ** 2, to_2d=True))
print(df2.vbt.apply(apply_func=lambda x: x ** 2))

i2
x2    1
y2    4
z2    9
Name: a2, dtype: int64
i2
x2    1
y2    4
z2    9
Name: a2, dtype: int64
c4  a4
i4    
x4   1
y4   4
z4   9


In [95]:
print(pd.DataFrame.vbt.concat(sr2, 10, df4, keys=['a', 'b', 'c']))
print(sr2.vbt.concat(10, df4, keys=['a', 'b', 'c']))

       a         b          c      
c6    a6 b6 c6  a6  b6  c6 a6 b6 c6
i2 i6                              
x2 x6  1  1  1  10  10  10  1  2  3
y2 y6  2  2  2  10  10  10  4  5  6
z2 z6  3  3  3  10  10  10  7  8  9
       a         b          c      
c6    a6 b6 c6  a6  b6  c6 a6 b6 c6
i2 i6                              
x2 x6  1  1  1  10  10  10  1  2  3
y2 y6  2  2  2  10  10  10  4  5  6
z2 z6  3  3  3  10  10  10  7  8  9


In [96]:
print(sr2.vbt.apply_and_concat(3, sr2.values, 10, apply_func=lambda i, x, y, c, d=1: x + y[i] + c + d, d=100))
print(sr2.vbt.apply_and_concat(3, sr2.values, 10, apply_func=njit(lambda i, x, y, c: x + y[i] + c + 100)))
print(sr2.vbt.apply_and_concat(3, df4.values, 10, apply_func=lambda i, x, y, c, d=1: x + y[:, i] + c + d, d=100))
print(sr2.vbt.apply_and_concat(3, df4.values, 10, apply_func=njit(lambda i, x, y, c: x + y[:, i] + c + 100)))
print(df4.vbt.apply_and_concat(3, df4.values, 10, apply_func=lambda i, x, y, c, d=1: x + y[:, i] + c + d, d=100))
print(df4.vbt.apply_and_concat(
    3, 
    df4.values, 
    10, 
    apply_func=njit(lambda i, x, y, c: x + y[:, i] + c + 100), 
    keys=pd.Index(['a', 'b', 'c'], name='hello')))

apply_idx    0    1    2
i2                      
x2         112  113  114
y2         113  114  115
z2         114  115  116
apply_idx    0    1    2
i2                      
x2         112  113  114
y2         113  114  115
z2         114  115  116
apply_idx    0    1    2
i2                      
x2         112  113  114
y2         116  117  118
z2         120  121  122
apply_idx    0    1    2
i2                      
x2         112  113  114
y2         116  117  118
z2         120  121  122
apply_idx    0              1              2          
c6          a6   b6   c6   a6   b6   c6   a6   b6   c6
i6                                                    
x6         112  116  120  113  117  121  114  118  122
y6         115  119  123  116  120  124  117  121  125
z6         118  122  126  119  123  127  120  124  128
hello    a              b              c          
c6      a6   b6   c6   a6   b6   c6   a6   b6   c6
i6                                                
x6     112  116  

In [97]:
print(sr2.vbt.combine(10., combine_func=lambda x, y: x + y))
print(sr2.vbt.combine(10, 100, d=1000, combine_func=lambda x, y, c, d=1: x + y + c + d)) # test args and kwargs
print(sr2.vbt.combine(np.array([10, 20, 30]), combine_func=lambda x, y: x + y))
print(sr2.vbt.combine(np.array([[10, 20, 30]]), combine_func=lambda x, y: x + y))
print(sr2.vbt.combine(sr1, combine_func=lambda x, y: x + y, broadcast_kwargs=dict(index_from='stack')))
print(sr2.vbt.combine(sr2, combine_func=lambda x, y: x + y, broadcast_kwargs=dict(index_from='stack')))
print(sr2.vbt.combine(df2, combine_func=lambda x, y: x + y, broadcast_kwargs=dict(index_from='stack')))
print(sr2.vbt.combine(df3, combine_func=lambda x, y: x + y, broadcast_kwargs=dict(index_from='stack')))
print(sr2.vbt.combine(df4, combine_func=lambda x, y: x + y, broadcast_kwargs=dict(index_from='stack')))
print(sr2.vbt.combine(df5, combine_func=lambda x, y: x + y, broadcast_kwargs=dict(index_from='stack')))

i2
x2    11.0
y2    12.0
z2    13.0
Name: a2, dtype: float64
i2
x2    1111
y2    1112
z2    1113
Name: a2, dtype: int64
i2
x2    11
y2    22
z2    33
Name: a2, dtype: int64
    a2  a2  a2
i2            
x2  11  21  31
y2  12  22  32
z2  13  23  33
i2  i1
x2  x1    2
y2  x1    3
z2  x1    4
dtype: int64
i2
x2    2
y2    4
z2    6
Name: a2, dtype: int64
c4     a4
i2 i4    
x2 x4   2
y2 y4   4
z2 z4   6
c5     a5  b5  c5
i2 i5            
x2 x5   2   3   4
y2 x5   3   4   5
z2 x5   4   5   6
c6     a6  b6  c6
i2 i6            
x2 x6   2   3   4
y2 y6   6   7   8
z2 z6  10  11  12
c7        a7  b7  c7
c8        a8  b8  c8
i2 i7 i8            
x2 x7 x8   2   3   4
y2 y7 y8   6   7   8
z2 z7 z8  10  11  12


In [98]:
print(sr2.vbt.combine(
    [10, [10, 20, 30], pd.Series([10, 20, 30])],
    10, b=100,
    combine_func=lambda x, y, a, b=1: x + y + a + b, 
    broadcast_kwargs=dict(index_from='stack')))
print(sr2.vbt.combine(
    [10, [10, 20, 30], [[10, 20, 30]], pd.Series([10, 20, 30]), df1, df3],
    10, b=100,
    combine_func=lambda x, y, a, b=1: x + y + a + b, 
    broadcast_kwargs=dict(index_from='stack')))
print(sr2.vbt.combine(
    [10, [10, 20, 30], [[10, 20, 30]], pd.Series([10, 20, 30]), df1, df3],
    10,
    combine_func=njit(lambda x, y, a, b=1: x + y + a + 100), 
    broadcast_kwargs=dict(index_from='stack')))
print(sr2.vbt.combine(
    [10, [10, 20, 30], [[10, 20, 30]], pd.Series([10, 20, 30]), df1, df3],
    10,
    combine_func=njit(lambda x, y, a, b=1: x + y + a + 100), 
    broadcast_kwargs=dict(index_from='stack')))

i2
x2    361
y2    382
z2    403
dtype: int64
c3         a3          
c5         a5   b5   c5
i2 i3 i5               
x2 x3 x5  703  724  745
y2 x3 x5  714  735  756
z2 x3 x5  725  746  767
c3         a3          
c5         a5   b5   c5
i2 i3 i5               
x2 x3 x5  703  724  745
y2 x3 x5  714  735  756
z2 x3 x5  725  746  767
c3         a3          
c5         a5   b5   c5
i2 i3 i5               
x2 x3 x5  703  724  745
y2 x3 x5  714  735  756
z2 x3 x5  725  746  767


In [99]:
# Test concat=True
print(sr2.vbt.combine(
    [10, [10, 20, 30], pd.Series([10, 20, 30])],
    10, b=100,
    combine_func=lambda x, y, a, b=1: x + y + a + b, 
    concat=True,
    broadcast_kwargs=dict(index_from='stack')))
print(sr2.vbt.combine(
    [10, [10, 20, 30], [[10, 20, 30]], pd.Series([10, 20, 30]), df1, df3],
    10, b=100,
    combine_func=lambda x, y, a, b=1: x + y + a + b, 
    concat=True,
    broadcast_kwargs=dict(index_from='stack')))
print(sr2.vbt.combine(
    [10, [10, 20, 30], [[10, 20, 30]], pd.Series([10, 20, 30]), df1, df3],
    10,
    combine_func=njit(lambda x, y, a, b=1: x + y + a + 100),
    concat=True,
    broadcast_kwargs=dict(index_from='stack')))
print(sr2.vbt.combine(
    [10, [10, 20, 30], [[10, 20, 30]], pd.Series([10, 20, 30]), df1, df3],
    10,
    combine_func=njit(lambda x, y, a, b=1: x + y + a + 100),
    concat=True,
    keys=['a', 'b', 'c', 'd', 'e', 'f'],
    broadcast_kwargs=dict(index_from='stack')))

combine_idx    0    1    2
i2                        
x2           121  121  121
y2           122  132  132
z2           123  143  143
combine_idx    0              1              2              3              4  \
c3            a3             a3             a3             a3             a3   
c5            a5   b5   c5   a5   b5   c5   a5   b5   c5   a5   b5   c5   a5   
i2 i3 i5                                                                       
x2 x3 x5     121  121  121  121  131  141  121  131  141  121  121  121  112   
y2 x3 x5     122  122  122  122  132  142  122  132  142  132  132  132  113   
z2 x3 x5     123  123  123  123  133  143  123  133  143  143  143  143  114   

combine_idx              5            
c3                      a3            
c5            b5   c5   a5   b5   c5  
i2 i3 i5                              
x2 x3 x5     112  112  112  113  114  
y2 x3 x5     113  113  113  114  115  
z2 x3 x5     114  114  114  115  116  
combine_idx    0              1

In [100]:
# Use magic methods with .vbt to do operations with custom broadcasting
print(pd.Series([1, 2, 3]).vbt + [1, 2, 3])
print(df3.vbt + df4.vbt)  # regular df3 + df4 will return nans

0    2
1    4
2    6
dtype: int64
c5    a5  b5  c5
c6    a6  b6  c6
i5 i6           
x5 x6  2   4   6
   y6  5   7   9
   z6  8  10  12
